### Install Dependencies

In [1]:
!pip install ultralytics opencv-python-headless ipywidgets tqdm --quiet



[notice] A new release of pip available: 22.3 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


### Imports

In [2]:
import cv2
from ultralytics import YOLO
import os
from ipywidgets import FileUpload, VBox, Button, Output
from IPython.display import display
from tqdm import tqdm


### Loading Model

In [ ]:
model_path = "model.pt"  # your trained model
model = YOLO(model_path)
device = "cpu"  # force CPU


### Processing Function

In [ ]:
def process_videos(b):
    with out:
        out.clear_output()
        output_folder = "processed_videos"
        os.makedirs(output_folder, exist_ok=True)

        if not upload.value:
            print("No videos uploaded!")
            return

        for fname, file_info in upload.value.items():
            print(f"Processing {fname}...")
            
            # Save uploaded video locally
            input_path = os.path.join(output_folder, fname)
            with open(input_path, "wb") as f:
                f.write(file_info['content'])

            # Prepare video I/O
            cap = cv2.VideoCapture(input_path)
            width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
            height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
            fps = cap.get(cv2.CAP_PROP_FPS)
            output_path = os.path.join(output_folder, f"processed_{fname}")
            fourcc = cv2.VideoWriter_fourcc(*"mp4v")
            out_vid = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

            # Process frames with progress bar
            total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            for _ in tqdm(range(total_frames)):
                ret, frame = cap.read()
                if not ret:
                    break
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                results = model.predict(frame_rgb, device=device, verbose=False)
                annotated_frame = results[0].plot()
                out_vid.write(cv2.cvtColor(annotated_frame, cv2.COLOR_RGB2BGR))

            cap.release()
            out_vid.release()
            print(f"Saved processed video: {output_path}\n")
        
        print("All uploaded videos processed!")

process_btn.on_click(process_videos)
